In [1]:
import _resume_eval_import_helper

In [41]:
from pathlib import Path
from tqdm.auto import tqdm
import json
import pandas as pd 
from pprint import pprint


In [34]:
job_description = pd.read_csv("output/filtered_job_description.csv")
talent_pool = pd.read_csv("output/filtered_talent_pool.csv")

# gpt-4o


In [21]:
# gpt-4o 

main_path = Path("output_20240831_1144")
json_files = list(main_path.glob("**/*.json"))
model_consistency = {}

for file in tqdm(json_files, desc="Processing GPT-4o files"):
    file_name = file.stem
    job_id, cv_id, _ = file_name.split("_")
    with open(file, "r") as f:
        data = json.load(f)
        if job_id not in model_consistency:
            model_consistency[job_id] = []
        else:
            model_consistency[job_id].append(data["job_description_analysis"])

Processing GPT-4o files:   0%|          | 0/704 [00:00<?, ?it/s]

In [80]:
compiled_jd = {}

for job_id, results in model_consistency.items():
    jd_nice_to_have = set()
    jd_essential = set()
    for analysis in results:
        essentials = [skill.lower() for skill in analysis["technical_skills"]["essential"]]
        jd_essential.add(tuple(sorted(essentials)))
        advantages = [skill.lower() for skill in analysis["technical_skills"]["advantageous"]]
        jd_nice_to_have.add(tuple(sorted(advantages)))
    compiled_jd[job_id] = {
        "jd_essential": jd_essential,
        "jd_nice_to_have": jd_nice_to_have
    }
    
compiled_jd = pd.DataFrame(compiled_jd).T
compiled_jd.reset_index(inplace=True)

In [81]:
compiled_jd["count_essential"] = compiled_jd["jd_essential"].apply(len)
compiled_jd["count_nice_to_have"] = compiled_jd["jd_nice_to_have"].apply(len)

compiled_jd.sort_values("count_essential", ascending=False).head(10)

,index,jd_essential,jd_nice_to_have,count_essential,count_nice_to_have
5,2f1247bd-4c98-4357-8b8b-ac52ee8698b2,"{(aws, linux, python, shell scripting, sql), (...","{(docker, microservices, mongodb), (aws,), (aw...",29,29
12,8f52ff7e-1929-49d9-86d3-052e98986b34,"{(.net, asp.net, aws, azure, c#, c++, css, ent...","{(angularjs, c++, css, docker, entity framewor...",25,20
2,7e259146-d9df-411e-8550-2fef51dc7bc9,"{(celery, devops, django, flask, git, mocking,...","{(aws, azure, data science, deep learning, geo...",19,5
4,dcb1513a-460d-46f7-81b1-23294a691bfd,"{(angular, css, docker, git, html 5, java, jav...","{(ant, bitbucket, cloud foundry, cucumber, ehc...",17,18
6,769c3093-32c5-4122-ae8f-d4f99a22354a,"{(angularjs, express.js, javascript, mongodb, ...","{(express.js, mongodb, mysql, react.js), (dock...",12,10
13,90c1ee72-9c8d-46d4-a671-cb7cba7f75df,"{(ansible, api gateway, aws, cloudformation, d...","{(ansible, cloudflare, grafana, kafka, prometh...",12,20
9,83d835b5-ab57-4044-ace8-a5d8ffc0e254,"{(auto layout, charts, cocoapods, debugging, d...","{(agile/scrum, github, jira, mentoring junior ...",9,3
8,91793843-4ca9-4eb6-9d84-8c59a3d19812,"{(angularjs, css, express.js, git, html, javas...","{(angular.js, asynchronous coding techniques, ...",5,3
14,66c5e115-cf87-4917-9cff-c2d72156cff6,"{(css3, html5, javascript, jquery, plugin crea...","{(), (page builders, woocommerce), (database i...",5,3
1,ca0539ba-835b-48fb-bed0-705d6668c372,"{(asp.net, c#, microsoft access, microsoft exc...","{(erp systems, wts paradigm), (erp systems, ss...",4,3


In [104]:
j = "7e259146-d9df-411e-8550-2fef51dc7bc9"  

print(job_description[job_description["Job ID"] == j]["Job Description"].values[0])

print("-"*100)

pprint(compiled_jd[compiled_jd["index"] == j]["jd_essential"].values[0])


Job Responsibilities:
6+ years of Hands-on experience in Python programming along with strong experience in SQL.
Experience in designing distributed systems using Python.
Experience in being a mentor for a team comprising of both experienced as well as fresh graduates who are python developers.
Experience with Python, along with other common libraries/frameworks such as Flask, Celery, Django, etc.
Basic understanding of developing private PIP packages.
Solid understanding of object-oriented programming.
Familiarity with concepts of MVC, Mocking, ORM, and REST APIs.
Experience in writing Unit Tests using frameworks like PyTest.
A knack for writing clean, readable Python code.
Ability to integrate multiple data sources and databases into one system.
Ability to understand monitoring metrics, such as CPU, Memory, Disk Space, Network latency, and telemetry services.
Understanding of fundamental design principles behind a scalable application.
Understanding the fundamental of RDBMS databases

In [101]:
j = "8f52ff7e-1929-49d9-86d3-052e98986b34"  

print(job_description[job_description["Job ID"] == j]["Job Description"].values[0])

print("-"*100)

print(compiled_jd[compiled_jd["index"] == j]["jd_essential"].values[0])


looking for application support engineer freshers can also apply
RESPONSIBILITIES:
Support of Application Stack hosted in AWS and Azure Cloud environments, on-premise and 3rd Party Data Centers.
Provide rotational and out of hours support as needed on an on-call basis
Handle testing, changes, deployment on dev, stage and production environment within SLAs
Identify new processes or modify existing processes to improve efficiencies in the support and monitoring of the department's Application suite
Assume responsibility for bug fix and small change requests on specified Applications
Additional development activities anticipated (NB: whilst development opportunities are anticipated once the team is established, this is NOT a development role)
Ensure that the team follow company standards and industry best practice approaches to Development activities
Continuously evaluates the design, functionality and performance of supported Applications and makes thoughtful and significant recommendati

# llama3


In [84]:
main_path = Path("output_20240831_0124")
json_files = list(main_path.glob("**/*.json"))
llama3_consistency = {}

for file in tqdm(json_files, desc="Processing llama3 files"):
    file_name = file.stem
    job_id, cv_id, _ = file_name.split("_")
    with open(file, "r") as f:
        data = json.load(f)
        if job_id not in llama3_consistency:
            llama3_consistency[job_id] = []
        else:
            llama3_consistency[job_id].append(data["job_description_analysis"])

Processing llama3 files:   0%|          | 0/705 [00:00<?, ?it/s]

In [93]:
llama3_compiled_jd = {}

for job_id, results in llama3_consistency.items():
    jd_nice_to_have = set()
    jd_essential = set()
    for analysis in results:
        essentials = [skill.lower() for skill in analysis["technical_skills"]["essential"]]
        jd_essential.add(tuple(sorted(essentials)))
        advantages = [skill.lower() for skill in analysis["technical_skills"]["advantageous"]]
        jd_nice_to_have.add(tuple(sorted(advantages)))
    llama3_compiled_jd[job_id] = {
        "jd_essential": jd_essential,
        "jd_nice_to_have": jd_nice_to_have
    }
    
llama3_compiled_jd = pd.DataFrame(llama3_compiled_jd).T
llama3_compiled_jd.reset_index(inplace=True)

In [95]:
llama3_compiled_jd["count_essential"] = llama3_compiled_jd["jd_essential"].apply(len)
llama3_compiled_jd["count_nice_to_have"] = llama3_compiled_jd["jd_nice_to_have"].apply(len)

llama3_compiled_jd.sort_values("count_essential", ascending=False).head(10)

,index,jd_essential,jd_nice_to_have,count_essential,count_nice_to_have
9,dcb1513a-460d-46f7-81b1-23294a691bfd,"{(angularjs, cloud deployments, cloud platform...","{(aws, azure, cloud foundry, docker, ehcache, ...",25,5
13,8f52ff7e-1929-49d9-86d3-052e98986b34,"{(.net development toolsets, .net web api, agi...","{(docker, kubernetes, python), (docker, entity...",23,2
4,7e259146-d9df-411e-8550-2fef51dc7bc9,"{(celery, django, flask, git, pytest, python, ...","{(aws, azure, deep learning, geo-spatial data,...",13,10
10,90c1ee72-9c8d-46d4-a671-cb7cba7f75df,"{(ansible, aws, cloudformation, docker, grafan...","{(active-mq, cloudflare cdn, kafka, qualys was...",13,4
2,83d835b5-ab57-4044-ace8-a5d8ffc0e254,"{(cocoapods, ios development, objective c, off...","{(aws,), (agile/scrum development methodologie...",9,6
12,b04cc5ce-b93e-427e-9439-0965e64779ff,"{(atlassian, css3, git, html5, javascript, jir...",{()},9,1
11,769c3093-32c5-4122-ae8f-d4f99a22354a,"{(mean, mern, stack), (mean, mern, sql), (mean...",{()},8,1
14,2f1247bd-4c98-4357-8b8b-ac52ee8698b2,"{(ecommerce application, mobile software, soft...","{(aws, ecommerce application, mobile software)...",8,5
0,5535b3b6-f919-4e04-bb23-5eb63436941f,"{(database development, db modeling, oracle, p...","{(agile development process, ba/da experience)...",7,5
7,91793843-4ca9-4eb6-9d84-8c59a3d19812,"{(angularjs, css, express.js, git, html, javas...",{()},7,1


In [97]:
j = "dcb1513a-460d-46f7-81b1-23294a691bfd"  

pprint(job_description[job_description["Job ID"] == j]["Job Description"].values[0])

print("-"*100)

pprint(compiled_jd[compiled_jd["index"] == j]["jd_essential"].values[0])


('Corporate Title – Associate Director\n'
 '\n'
 'Functional Role - Principal Software Engineer\n'
 '\n'
 'Are you ready to explore a world of possibilities?\n'
 '\n'
 'Join our DTCC family, and you’ll grow your expertise and become the best '
 'version of you. As you embark on a new journey, you’ll tackle challenges '
 'with flexibility and grace, learning new skills and advancing your career '
 'while having the time of your life.\n'
 '\n'
 'Why You Would Love This Job:\n'
 '\n'
 'Being a member of the SIFMU IT team means you will be a part of a technology '
 'team with a rich diverse skill sets and a great enthusiastic committed team. '
 'Whether it’s Java, Spring suite, JavaScript, AngularJS, Unix or Database '
 'skillset required for the project initiatives, we are there for each other '
 'collaborating and helping each other to achieve the common goal. We are '
 'embarking on an incredible multi-year transformation journey and we are '
 'looking for best-of-breed software enginee

In [98]:
print(job_description[job_description["Job ID"] == j]["Job Description"].values[0])

Corporate Title – Associate Director

Functional Role - Principal Software Engineer

Are you ready to explore a world of possibilities?

Join our DTCC family, and you’ll grow your expertise and become the best version of you. As you embark on a new journey, you’ll tackle challenges with flexibility and grace, learning new skills and advancing your career while having the time of your life.

Why You Would Love This Job:

Being a member of the SIFMU IT team means you will be a part of a technology team with a rich diverse skill sets and a great enthusiastic committed team. Whether it’s Java, Spring suite, JavaScript, AngularJS, Unix or Database skillset required for the project initiatives, we are there for each other collaborating and helping each other to achieve the common goal. We are embarking on an incredible multi-year transformation journey and we are looking for best-of-breed software engineers to join us on this journey.

In this role would be responsible to contribute towards 

In [108]:
talent_pool.iloc[1]["Resume"]

'Education Details \nJune 2013 to June 2016 Diploma Computer science Pune, Maharashtra Aissms\nJune 2016 BE pursuing Computer science Pune, Maharashtra Anantrao pawar college of Engineering & Research centre\nPython Developer \nSkill Details \nCompany Details \ncompany - Cybage Software Pvt. Ltd\ndescription - I want to work in organisation as a python developer to utilize my knowledge & To gain more knowledge with our organisation.'